<a href="https://colab.research.google.com/github/sj0054pe/MBTI_Estim/blob/master/src/5A_S12_BERT%E7%94%A8%E3%83%87%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%83%E3%83%88_U20%E6%AD%A3%E8%A6%8F%E8%A1%A8%E7%8F%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center>BERT データセットの作成 S12</center>

###<center>リツイート・@mention・URLの除去・短い文(20単語未満)の削除</center>

###<center>botを除去 ・ @mention名も正規表現で確実に除去</center>


#0. 作業環境の準備

##0.1. google driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##0.2. カレントディレクトリへ移動

In [ ]:
%cd /content/drive/MyDrive/研究室/研究_M2/src

/content/drive/MyDrive/研究室/研究_M2/src


##0.3. 現在時刻の取得

In [ ]:
import datetime
import pytz

def pull_Datetime():
  DATETIME_now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
  DATETIME_now = DATETIME_now.strftime("%Y-%m-%d")
  #DATETIME_now = DATETIME_now.strftime("%Y/%m/%d %H:%M:%S")
  #print(DATETIME_now)

  return DATETIME_now

print(pull_Datetime())

2022-01-25


In [ ]:
SEASONS='S12'
HIGHER ='20'
RAFFLE=30000

TERM='%s_W%s_R%s' % (SEASONS, HIGHER, RAFFLE)
print(TERM)

S12_W20_R30000


##option. データセットの指定

In [ ]:
INPUT_CSV_v5='./Assets/Assets_Output/Tweet_Datasets/v7_all_2021-12-04_W20,csv'
INPUT_CSV_v5='./Assets/Assets_Output/Tweet_Datasets/v7_all_2021-12-04_W20.csv'

#1. データセットのロード

In [ ]:
import pandas as pd
df_v0=pd.read_csv('%s' % INPUT_CSV_v5, engine='python')

# データの確認
print(f'データサイズ： {df_v0.shape}')

df_v0[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']]

データサイズ： (133350, 14)


,text,tweet_time,MBTI_Type,label_EorI,label_NorS,label_ForT,label_JorP
0,CLIP STUDIO TABMATEがコミケで販売されたそうだけど買った方いるのかな！？,2017-12-29 14:01:24+00:00,INTJ,I,N,T,J
1,中小の劇団のチケットって、安すぎたら箱や演者やスタッフさんのギャラが払えない。かと言って高く...,2017-11-26 06:54:54+00:00,INTJ,I,N,T,J
2,Vステなら大丈夫だったのになぁ… まぁQちゃんの事だからなんとかなるでしょう。,2017-11-19 16:00:34+00:00,INTJ,I,N,T,J
3,"こんなにカッコよくキメてたのに…#buzzup BOOSTER - KEN THE 390,...",2017-11-09 14:32:30+00:00,INTJ,I,N,T,J
4,ブーメランパンツはあなたの息子と一緒にクリスマスの一夜を過ごします。可愛がってくださいね。 ...,2017-11-07 16:36:08+00:00,INTJ,I,N,T,J
...,...,...,...,...,...,...,...
133345,ずっと思ってたんだけど、 紅蓮でスキル一新されたのは良いけど、旧コンテンツで使ってたスキル...,2017-08-18 23:24:04+00:00,ESFP,E,S,F,P
133346,肩と背中が痛いと言ったらうちの子に肩たたきをしてもらった ほっこり,2017-08-14 19:01:58+00:00,ESFP,E,S,F,P
133347,もう無理なんでそんな元気なんこれから仕事行って第2ラウンドカラオケとかもうどうなってるんその体,2017-08-14 03:47:40+00:00,ESFP,E,S,F,P
133348,コンタクトどこ行った？ →私「まさか捨てた？」 →捨ててたわ ていうかマンガみたいな会話を...,2017-08-13 20:42:00+00:00,ESFP,E,S,F,P


#2. データの加工

##2.1. 必要なデータ（文章とラベル）の抽出

In [ ]:
#df_v0.drop(['notes', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1',  'Unnamed: 0.1.1.1'],axis=1, inplace=True)
df_v0.dropna(how='any', axis=0, inplace=True)

df_v0=df_v0
df_v0[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']]

,text,tweet_time,MBTI_Type,label_EorI,label_NorS,label_ForT,label_JorP
0,CLIP STUDIO TABMATEがコミケで販売されたそうだけど買った方いるのかな！？,2017-12-29 14:01:24+00:00,INTJ,I,N,T,J
1,中小の劇団のチケットって、安すぎたら箱や演者やスタッフさんのギャラが払えない。かと言って高く...,2017-11-26 06:54:54+00:00,INTJ,I,N,T,J
2,Vステなら大丈夫だったのになぁ… まぁQちゃんの事だからなんとかなるでしょう。,2017-11-19 16:00:34+00:00,INTJ,I,N,T,J
3,"こんなにカッコよくキメてたのに…#buzzup BOOSTER - KEN THE 390,...",2017-11-09 14:32:30+00:00,INTJ,I,N,T,J
4,ブーメランパンツはあなたの息子と一緒にクリスマスの一夜を過ごします。可愛がってくださいね。 ...,2017-11-07 16:36:08+00:00,INTJ,I,N,T,J
...,...,...,...,...,...,...,...
133345,ずっと思ってたんだけど、 紅蓮でスキル一新されたのは良いけど、旧コンテンツで使ってたスキル...,2017-08-18 23:24:04+00:00,ESFP,E,S,F,P
133346,肩と背中が痛いと言ったらうちの子に肩たたきをしてもらった ほっこり,2017-08-14 19:01:58+00:00,ESFP,E,S,F,P
133347,もう無理なんでそんな元気なんこれから仕事行って第2ラウンドカラオケとかもうどうなってるんその体,2017-08-14 03:47:40+00:00,ESFP,E,S,F,P
133348,コンタクトどこ行った？ →私「まさか捨てた？」 →捨ててたわ ていうかマンガみたいな会話を...,2017-08-13 20:42:00+00:00,ESFP,E,S,F,P


In [ ]:
df_v3 = df_v0
#df_v3=df.sample(n=100)
df_v3[['text', 'tweet_time', 'conversation_id','MBTI_Type']].head(15)

#df_v3['text'].head(5)

,text,tweet_time,conversation_id,MBTI_Type
0,CLIP STUDIO TABMATEがコミケで販売されたそうだけど買った方いるのかな！？,2017-12-29 14:01:24+00:00,9.467430e+17,INTJ
1,中小の劇団のチケットって、安すぎたら箱や演者やスタッフさんのギャラが払えない。かと言って高く...,2017-11-26 06:54:54+00:00,9.346768e+17,INTJ
2,Vステなら大丈夫だったのになぁ… まぁQちゃんの事だからなんとかなるでしょう。,2017-11-19 16:00:34+00:00,9.322774e+17,INTJ
3,"こんなにカッコよくキメてたのに…#buzzup BOOSTER - KEN THE 390,...",2017-11-09 14:32:30+00:00,9.286314e+17,INTJ
4,ブーメランパンツはあなたの息子と一緒にクリスマスの一夜を過ごします。可愛がってくださいね。 ...,2017-11-07 16:36:08+00:00,9.279377e+17,INTJ
5,ブーメランパンツのココアは爆発しました。 #ココア召喚チャレンジ 爆裂魔法ばりに爆発しました。,2017-11-07 16:34:08+00:00,9.279372e+17,INTJ
6,U-boat #ゆと打って最初に出てきたのがお前の武器 また潜水艦かよ！ いや好きだけど...,2017-10-27 17:48:17+00:00,9.239696e+17,INTJ
7,あとちょっとズルい方法ですが、ヤバイかな〜って段階になったら、上司なり先輩なりに「この方法で...,2017-10-24 15:04:32+00:00,9.228265e+17,INTJ
8,大抵の人もミスして皆それぞれ自分のやりやすい方法にカスタマイズしてるのに、それをすっ飛ばして...,2017-10-24 14:25:37+00:00,9.228265e+17,INTJ
9,ブーメランパンツさんは「京風スケベ」です。 友人が心配してますよ。 #あなたのスケベのジャ...,2017-10-24 10:49:57+00:00,9.227772e+17,INTJ


In [ ]:
import re
import numpy as np

def remove_bot(text):
  list_text=re.split(' |\n', text)
  #print(list_text)
  len_list=len(list_text)
  Nan = np.nan
  count_element=0
  element_URL=None
  element_bot=None

  judge_bot='No'
  for idx, element in enumerate(list_text):
    if re.match(r'(【)[\S]*(】)*', element):
      print(element)
      judge_bot='Yes'
    # if re.match(r'^https?://([\w-]+\.)+[\w-]+(/[\w-./?%&=]*)?$', element):
    #   print(element)
    #   element_URL=element
      
  #print(element_mention, element_URL)
  if judge_bot=='No':
    Z=' '.join(list_text)
  elif judge_bot=='Yes':
    Z=Nan
  return Z

  # if element_URL==None:
  #   pass
  # else:
  #   list_text.remove(element_URL)

  # Z=' '.join(list_text)
  # return Z

df_v3['text'] = df_v3['text'].map(remove_bot)
df_v3.dropna(how='any', axis=0, inplace=True)

df_v3['text']

ストリーミング出力は最後の 5000 行に切り捨てられました。
【教訓】
【
【検証:どうぶつの森】着替え時間“0.3秒”は本当に最速か!?
【人気爆発】可愛すぎるバーチャルYouTuberキズナアイのおすすめ動画ランキングベスト10
【共謀罪対抗策】ふと思ったんだが…共謀情報を溢れさせればいいんじゃね？例えば「独裁＆縁故主義政治を打倒する強力な手段について、いつどこで話し合いを」てなツイートをみんな発信しまくる。もし何万って人が毎日これをやれば…。強権を手にした連中があたふたする様を想像するのは少し愉しいぞ。
【最近よく聴いてる音楽】キース・ジャレット（70年代まで）、チャールズ・ロイド、ビル・フリゼールなど。なんかそんな気分なんだな。
【最近読んだ本】田中ロミオ『犬と魔法のファンタジー』。“魔法”“冒険”というラノベのテンプレを借りた教養小説、なのだと思う。そういう意味で、いわゆる普通のラノベとは一線を画す作品。僕は大好きだけど、ファンタジー的ワクワク感は皆無なので、読むときは覚悟が必要かも（笑）。
【最近読んだ本】なだいなだ『人間、この非人間的なもの』。面白かった。「ケシカランものを排除し、穢れを浄め、純粋さを求める、そういった感覚的正義感」が、どれだけ全体主義に力を与えるかが分かりやすく述べられている。この本の内容がとても今日的に感じられるのは、悲しいことではあるけれど。
【共謀罪】この記事（とくに4P目）を読んでると、「肉屋を支持する豚」ってフレーズを思い出す。あと『下ネタという概念が存在しない退屈な世界』も（笑）。異論という概念を承認しない窮屈な世界の住民たちは何を思うだろう？
【ルーマー「アレサ」】クラシック化してるのはロックだけじゃなくて、70sポップンソウル的なのもそうなんだと思う。そういうのって現象としてはつまんないけど、音楽自体にはやっぱイイネって反応してしまう。対訳付きで観ると、ほんと泣けるわ。
【忖度社会】「空気を読む」って、もうすっかり当たり前のマナーみたいになってるけど、これがさらに進んで、「顔色を伺う」→「長いものに巻かれる」が当たり前になって、しまいに「赤信号みんなで渡れば怖くない」でもって大事故に～！みたいなことになったらイヤだなぁと思う昨今。
【森友学園】事件の背景として、1.政権の周りに「極右こそ正義」的な原理ができあがる　2

0             CLIP STUDIO TABMATEがコミケで販売されたそうだけど買った方いるのかな！？
1         中小の劇団のチケットって、安すぎたら箱や演者やスタッフさんのギャラが払えない。かと言って高く...
2                   Vステなら大丈夫だったのになぁ… まぁQちゃんの事だからなんとかなるでしょう。
4         ブーメランパンツはあなたの息子と一緒にクリスマスの一夜を過ごします。可愛がってくださいね。 ...
5           ブーメランパンツのココアは爆発しました。 #ココア召喚チャレンジ 爆裂魔法ばりに爆発しました。
                                ...                        
133345    ずっと思ってたんだけど、  紅蓮でスキル一新されたのは良いけど、旧コンテンツで使ってたスキル...
133346                   肩と背中が痛いと言ったらうちの子に肩たたきをしてもらった  ほっこり
133347      もう無理なんでそんな元気なんこれから仕事行って第2ラウンドカラオケとかもうどうなってるんその体
133348    コンタクトどこ行った？ →私「まさか捨てた？」 →捨ててたわ  ていうかマンガみたいな会話を...
133349    お薬スープ200、角煮100、お寿司100、 ……仕込みが終わった。これでせめて１日位もってくれ。
Name: text, Length: 128606, dtype: object

In [ ]:
print(f'データサイズ： {df_v3.shape}')

データサイズ： (128606, 14)


In [ ]:
df_v3A=df_v3

import re
import numpy as np

def remove_mention(text):
  # print('--------------')
  #print(text)
  list_text=re.split(' |\n', text)
  #print(list_text)
  len_list=len(list_text)
  Nan = np.nan
  count_element=0
  # print(key_hashtag)
  # print(list_text)
  judge_bot='No'
  for idx, element in enumerate(list_text):
    element=element.rstrip()
    #print(element)
    #if re.search(r'%s' % key_hashtag, element):
    if element.startswith('@'):
      print(element)
      list_text[idx]=''

  Z=' '.join(list_text)
  return Z

    # if re.match(r'^https?://([\w-]+\.)+[\w-]+(/[\w-./?%&=]*)?$', element):
    #   print(element)
    #   element_URL=element

  # if element_URL==None:
  #   pass
  # else:
  #   list_text.remove(element_URL)

  # Z=' '.join(list_text)
  # return Z

df_v3A.dropna(how='any', axis=0, inplace=True)
df_v3A['text']= df_v3A['text'].map(remove_mention)
df_v3A.dropna(how='any', axis=0, inplace=True)

# df_v3B['text']
print(f'データサイズ： {df_v3A.shape}')

@ocha2_mtar
@zw_yh　さん、Askart
@_326r　さん、さかいめかなえ
@loosescrew908
@Wildlife_daily
@hachitaro
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@Aquatic_girls
@stella_cat222
@clarinet_0921
@clap_clap_hand
@et_tan_rin4ring
@et_tan_rin4ring
@PfmeJR
@hideko44887292
@krkrft3337
@moso60ki
@GattoNero0126
@uhed_unit_507
@Miuchuchu1113
@porterrobinson
@UltraJapan
@UltraMusicjp
@tyuunibyouwww1
@YamadaDdon
@YamadaDdon
@1042banana
@_aeoe
@MICOOUGEE
@mikkun2133
@

In [ ]:
df_v3B=df_v3A

key_hashtag = '#twimaker'

import re
import numpy as np
print(key_hashtag)
def remove_bot_by_hashtag(text):
  # print('--------------')
  #print(text)
  list_text=re.split(' |\n', text)
  #print(list_text)
  len_list=len(list_text)
  Nan = np.nan
  count_element=0
  # print(key_hashtag)
  # print(list_text)
  judge_bot='No'
  for idx, element in enumerate(list_text):
    element=element.rstrip()
    #print(element)
    #if re.search(r'%s' % key_hashtag, element):
    if element.startswith('#'):
      if element== key_hashtag:
        # print(element)
        judge_bot = 'Yes'

  if judge_bot=='No':
    Z=' '.join(list_text)
  elif judge_bot=='Yes':
    Z=Nan
  return Z

    # if re.match(r'^https?://([\w-]+\.)+[\w-]+(/[\w-./?%&=]*)?$', element):
    #   print(element)
    #   element_URL=element

  # if element_URL==None:
  #   pass
  # else:
  #   list_text.remove(element_URL)

  # Z=' '.join(list_text)
  # return Z

df_v3B.dropna(how='any', axis=0, inplace=True)
df_v3B['text']= df_v3B['text'].map(remove_bot_by_hashtag)
df_v3B.dropna(how='any', axis=0, inplace=True)

# df_v3B['text']
print(f'データサイズ： {df_v3B.shape}')

#twimaker
データサイズ： (127712, 14)


In [ ]:
df_v3C=df_v3B

#key_hashtag = '#twimaker'
key_hashtag = '#SleepMeister'

import re
import numpy as np
print(key_hashtag)
def remove_bot_by_hashtag(text):
  # print('--------------')
  #print(text)
  list_text=re.split(' |\n', text)
  #print(list_text)
  len_list=len(list_text)
  Nan = np.nan
  count_element=0
  # print(key_hashtag)
  # print(list_text)
  judge_bot='No'
  for idx, element in enumerate(list_text):
    element=element.rstrip()
    #print(element)
    #if re.search(r'%s' % key_hashtag, element):
    if element.startswith('#'):
      if element== key_hashtag:
        # print(element)
        judge_bot = 'Yes'

  if judge_bot=='No':
    Z=' '.join(list_text)
  elif judge_bot=='Yes':
    Z=Nan
  return Z

    # if re.match(r'^https?://([\w-]+\.)+[\w-]+(/[\w-./?%&=]*)?$', element):
    #   print(element)
    #   element_URL=element
      
  #print(element_mention, element_URL)
  if element_bot==None:
    Z=' '.join(list_text)
  else:
    Z=Nan
  return Z

  # if element_URL==None:
  #   pass
  # else:
  #   list_text.remove(element_URL)

  # Z=' '.join(list_text)
  # return Z

df_v3C.dropna(how='any', axis=0, inplace=True)
df_v3C['text']= df_v3C['text'].map(remove_bot_by_hashtag)
df_v3C.dropna(how='any', axis=0, inplace=True)

# df_v3B['text']
print(f'データサイズ： {df_v3C.shape}')

#SleepMeister
データサイズ： (127646, 14)


In [ ]:
df_v3C.head(15)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,author_id,text,tweet_id,tweet_time,conversation_id,lang,MBTI_Type,label_EorI,label_NorS,label_ForT,label_JorP
0,1,1,11,8.726092e+07,CLIP STUDIO TABMATEがコミケで販売されたそうだけど買った方いるのかな！？,9.467430e+17,2017-12-29 14:01:24+00:00,9.467430e+17,ja,INTJ,I,N,T,J
1,2,2,100,2.888665e+09,中小の劇団のチケットって、安すぎたら箱や演者やスタッフさんのギャラが払えない。かと言って高く...,9.346768e+17,2017-11-26 06:54:54+00:00,9.346768e+17,ja,INTJ,I,N,T,J
2,3,3,104,2.888665e+09,Vステなら大丈夫だったのになぁ… まぁQちゃんの事だからなんとかなるでしょう。,9.322774e+17,2017-11-19 16:00:34+00:00,9.322774e+17,ja,INTJ,I,N,T,J
4,17,17,156,2.888665e+09,ブーメランパンツはあなたの息子と一緒にクリスマスの一夜を過ごします。可愛がってくださいね。 ...,9.279377e+17,2017-11-07 16:36:08+00:00,9.279377e+17,ja,INTJ,I,N,T,J
5,18,18,157,2.888665e+09,ブーメランパンツのココアは爆発しました。 #ココア召喚チャレンジ 爆裂魔法ばりに爆発しました。,9.279372e+17,2017-11-07 16:34:08+00:00,9.279372e+17,ja,INTJ,I,N,T,J
6,20,20,163,2.888665e+09,U-boat #ゆと打って最初に出てきたのがお前の武器 また潜水艦かよ！ いや好きだけど...,9.239696e+17,2017-10-27 17:48:17+00:00,9.239696e+17,ja,INTJ,I,N,T,J
7,21,21,165,2.888665e+09,あとちょっとズルい方法ですが、ヤバイかな〜って段階になったら、上司なり先輩なりに「この方法で...,9.228412e+17,2017-10-24 15:04:32+00:00,9.228265e+17,ja,INTJ,I,N,T,J
8,22,22,167,2.888665e+09,大抵の人もミスして皆それぞれ自分のやりやすい方法にカスタマイズしてるのに、それをすっ飛ばして...,9.228314e+17,2017-10-24 14:25:37+00:00,9.228265e+17,ja,INTJ,I,N,T,J
9,23,23,168,2.888665e+09,ブーメランパンツさんは「京風スケベ」です。 友人が心配してますよ。 #あなたのスケベのジャ...,9.227772e+17,2017-10-24 10:49:57+00:00,9.227772e+17,ja,INTJ,I,N,T,J
10,24,24,170,2.888665e+09,なんですと！青髪美少女が！！ しかも人間じゃないからナニをしても法律は適応されないからしたい...,9.226765e+17,2017-10-24 04:09:49+00:00,9.224387e+17,ja,INTJ,I,N,T,J


In [ ]:
!pip install janome

     |████████████████████████████████| 19.7 MB 1.2 MB/s 


In [ ]:
df_v3D=df_v3C

from janome.tokenizer import Tokenizer
import collections
from tqdm.notebook import tqdm

import re
import numpy as np

t = Tokenizer()

def remove_ShortText(text):
    one_tweet = "".join(text) #"\n".join(text)
    # print("----------------------------------------")
    # print(one_tweet)
    # print('↓')

    Nan = np.nan
    if len(list(t.tokenize(one_tweet)))> 19:
        #print("長い : ", one_tweet)
        #pass
        return one_tweet
    else:
        #print("短い : ", one_tweet)
        return None

# remove_ShortText(text)
# print()
#QQQ= df_v3['text'][1:100].map(remove_ShortText)
df_v3D['text'] = df_v3D['text'][:].map(remove_ShortText)
df_v3D.dropna(how='any', axis=0, inplace=True)

#df_v3D['text']
print(f'データサイズ： {df_v3D.shape}')

データサイズ： (127382, 14)


In [ ]:
df_v5=df_v3D

df_v5_E_1=df_v5[df_v5['label_EorI']=='E']
df_v5_I_1=df_v5[df_v5['label_EorI']=='I']
df_v5_E_2=df_v5_E_1.sample(n=15000)
df_v5_I_2=df_v5_I_1.sample(n=15000)
df_v5_EorI=pd.concat([df_v5_E_2, df_v5_I_2])

df_v5_N_1=df_v5[df_v5['label_NorS']=='N']
df_v5_S_1=df_v5[df_v5['label_NorS']=='S']
df_v5_N_2=df_v5_N_1.sample(n=15000)
df_v5_S_2=df_v5_S_1.sample(n=15000)
df_v5_NorS=pd.concat([df_v5_N_2, df_v5_S_2])

df_v5_F_1=df_v5[df_v5['label_ForT']=='F']
df_v5_T_1=df_v5[df_v5['label_ForT']=='T']
df_v5_F_2=df_v5_F_1.sample(n=15000)
df_v5_T_2=df_v5_T_1.sample(n=15000)
df_v5_ForT=pd.concat([df_v5_F_2, df_v5_T_2])

df_v5_J_1=df_v5[df_v5['label_JorP']=='J']
df_v5_P_1=df_v5[df_v5['label_JorP']=='P']
df_v5_J_2=df_v5_J_1.sample(n=15000)
df_v5_P_2=df_v5_P_1.sample(n=15000)
df_v5_JorP=pd.concat([df_v5_J_2, df_v5_P_2])

In [ ]:
# def num_Type_Index(element):
#   num_Users_Type=df_v5_EorI.groupby(['%s' % element])
#   num_Users_Type.groups
#   num_Users_Type.size()

#   print(num_Users_Type.size())
#   print()

# for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
#   num_Type_Index(element)
num_Users_Type=df_v5_EorI.groupby(['label_EorI'])
num_Users_Type.groups
num_Users_Type.size()
print(num_Users_Type.size())
print()
num_Users_Type=df_v5_NorS.groupby(['label_NorS'])
num_Users_Type.groups
num_Users_Type.size()
print(num_Users_Type.size())
print()
num_Users_Type=df_v5_ForT.groupby(['label_ForT'])
num_Users_Type.groups
num_Users_Type.size()
print(num_Users_Type.size())
print()
num_Users_Type=df_v5_JorP.groupby(['label_JorP'])
num_Users_Type.groups
num_Users_Type.size()
print(num_Users_Type.size())
print()

label_EorI
E    15000
I    15000
dtype: int64

label_NorS
N    15000
S    15000
dtype: int64

label_ForT
F    15000
T    15000
dtype: int64

label_JorP
J    15000
P    15000
dtype: int64



#3. データの保存

##3.1. 手法4までで作成した投稿を更新

##3.2. データセットS3の作成

INPUT_CSV_v7='./Assets/Assets_Output/Tweet_Datasets/v7S1_forBERT_2021-12-04_W20R10000.csv'

In [ ]:
theDate=pull_Datetime()
df_v5_all=df_v5
df_v5_all.to_csv('./Assets/Assets_Output/Tweet_Datasets/v6_all_forBERT_%s_%s.csv' % (theDate, HIGHER))
print('./Assets/Assets_Output/Tweet_Datasets/v6_all_forBERT_%s_%s.csv' % (theDate, HIGHER))
print()

df_v5_sample=df_v5_all.sample(n=10000)
df_v5_sample.to_csv('./Assets/Assets_Output/Tweet_Datasets/v7S1_forBERT_%s_W%sR10000.csv' % (theDate, HIGHER))
print('./Assets/Assets_Output/Tweet_Datasets/v7S1_forBERT_%s_W%sR10000.csv' % (theDate, HIGHER))
print()

./Assets/Assets_Output/Tweet_Datasets/v6_all_forBERT_2022-01-25_20.csv

./Assets/Assets_Output/Tweet_Datasets/v7S1_forBERT_2022-01-25_W20R10000.csv



In [ ]:
theDate=pull_Datetime()
df_v5_EorI.to_csv('./Assets/Assets_Output/Tweet_Datasets/v6%s_forBERT_%s_%s_%s_EorI.csv' % (SEASONS, theDate, HIGHER, RAFFLE))
print('./Assets/Assets_Output/Tweet_Datasets/v6%s_forBERT_%s_%s_%s_EorI.csv' % (SEASONS, theDate, HIGHER, RAFFLE))
print()
df_v5_NorS.to_csv('./Assets/Assets_Output/Tweet_Datasets/v6%s_forBERT_%s_%s_%s_NorS.csv' % (SEASONS, theDate, HIGHER, RAFFLE))
print('./Assets/Assets_Output/Tweet_Datasets/v6%s_forBERT_%s_%s_%s_NorS.csv' % (SEASONS, theDate, HIGHER, RAFFLE))
print()
df_v5_ForT.to_csv('./Assets/Assets_Output/Tweet_Datasets/v6%s_forBERT_%s_%s_%s_ForT.csv' % (SEASONS, theDate, HIGHER, RAFFLE))
print('./Assets/Assets_Output/Tweet_Datasets/v6%s_forBERT_%s_%s_%s_EorI.csv' % (SEASONS, theDate, HIGHER, RAFFLE))
print()
df_v5_JorP.to_csv('./Assets/Assets_Output/Tweet_Datasets/v6%s_forBERT_%s_%s_%s_JorP.csv' % (SEASONS, theDate, HIGHER, RAFFLE))
print('./Assets/Assets_Output/Tweet_Datasets/v6%s_forBERT_%s_%s_%s_JorP.csv' % (SEASONS, theDate, HIGHER, RAFFLE))
print()

./Assets/Assets_Output/Tweet_Datasets/v6S12_forBERT_2022-01-25_20_30000_EorI.csv

./Assets/Assets_Output/Tweet_Datasets/v6S12_forBERT_2022-01-25_20_30000_NorS.csv

./Assets/Assets_Output/Tweet_Datasets/v6S12_forBERT_2022-01-25_20_30000_EorI.csv

./Assets/Assets_Output/Tweet_Datasets/v6S12_forBERT_2022-01-25_20_30000_JorP.csv



##3.3. 記録用に各指標のユーザー数を表示

In [ ]:
def num_Type_Index(element):
  num_Users_Type=df_v5_EorI.groupby(['%s' % element])
  num_Users_Type.groups
  num_Users_Type.size()
  print(num_Users_Type.size())
  print()
for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_Type_Index(element)


def num_Type_Index(element):
  num_Users_Type=df_v5_NorS.groupby(['%s' % element])
  num_Users_Type.groups
  num_Users_Type.size()
  print(num_Users_Type.size())
  print()
for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_Type_Index(element)

def num_Type_Index(element):
  num_Users_Type=df_v5_ForT.groupby(['%s' % element])
  num_Users_Type.groups
  num_Users_Type.size()
  print(num_Users_Type.size())
  print()
for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_Type_Index(element)

def num_Type_Index(element):
  num_Users_Type=df_v5_JorP.groupby(['%s' % element])
  num_Users_Type.groups
  num_Users_Type.size()
  print(num_Users_Type.size())
  print()

for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_Type_Index(element)

label_EorI
E    15000
I    15000
dtype: int64

label_NorS
N    16503
S    13497
dtype: int64

label_ForT
F    19041
T    10959
dtype: int64

label_JorP
J    10915
P    19085
dtype: int64

label_EorI
E     4865
I    25135
dtype: int64

label_NorS
N    15000
S    15000
dtype: int64

label_ForT
F    20384
T     9616
dtype: int64

label_JorP
J    11243
P    18757
dtype: int64

label_EorI
E     5139
I    24861
dtype: int64

label_NorS
N    17952
S    12048
dtype: int64

label_ForT
F    15000
T    15000
dtype: int64

label_JorP
J    10451
P    19549
dtype: int64

label_EorI
E     4768
I    25232
dtype: int64

label_NorS
N    17039
S    12961
dtype: int64

label_ForT
F    20399
T     9601
dtype: int64

label_JorP
J    15000
P    15000
dtype: int64



#Option 最大単語数の計算

In [ ]:
!pip install transformers

!pip install fugashi
!pip install ipadic

In [ ]:
## Tokenizerの準備
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

In [ ]:
from tqdm.notebook import tqdm
import numpy as np

def num_max_words(TYPE_INDEX):
  sentences= df_S['text'].values #sentences = df.sentence.values
  labels= df_S['%s' % TYPE_INDEX].values #labels = df.label.values

  # 最大単語数の確認
  max_len = []
  # 1文づつ処理
  len_sentences = len(sentences)
  for sent in tqdm(sentences):
      # Tokenizeで分割
      token_words = tokenizer.tokenize(sent)
      # 文章数を取得してリストへ格納
      max_len.append(len(token_words))
  # 最大の値を確認
  len_max=max(max_len)
  #len_q0, len_q25, len_q50, len_q75, len_q100=np.percentile(max_len,  [0, 25, 50, 75, 100])
  #len_q75=int(len_q75)
  print('最大単語数: ', max(max_len))

for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_max_words(element)